# HuggingFace Model → RunPod Network Volume

Download models from HuggingFace on Colab and upload to RunPod Network Volume via S3-compatible API.

## Settings

Edit the Settings cell below:

- `HF_MODELS`: HuggingFace model names (`USER/REPOSITORY` format, e.g., `Qwen/Qwen3-8B`)
- `REGION`, `ENDPOINT_URL`, `BUCKET`: Get from https://console.runpod.io/user/storage "S3 Compatible API Commands"
  ```
  aws s3 ls --region xxx --endpoint-url https://s3api-xxx.runpod.io s3://your-volume-id/
  ```

## Colab Secrets

1. Click the 🔑 icon in the left sidebar
2. Add the following secrets:
   - `HF_TOKEN`: HuggingFace access token
   - `RUNPOD_STORAGE_ACCESS_KEY_ID`: RunPod Storage Access Key ID
   - `RUNPOD_STORAGE_SECRET_ACCESS_KEY`: RunPod Storage Secret Access Key
3. Enable "Notebook access" for each secret

In [ ]:
# Settings

# HuggingFace models (USER/REPOSITORY format, multiple allowed)
HF_MODELS = [
    "Qwen/Qwen3-8B",
]

# RunPod Storage (copy from https://console.runpod.io/user/storage)
REGION = ""  # @param {type:"string"}
ENDPOINT_URL = ""  # @param {type:"string"}
BUCKET = ""  # @param {type:"string"}

In [ ]:
# Environment Setup (run once)
import os
from google.colab import userdata

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('RUNPOD_STORAGE_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('RUNPOD_STORAGE_SECRET_ACCESS_KEY')

In [ ]:
# Download and upload each model
for model in HF_MODELS:
    local_dir = f"/content/models/{model}"
    dest_path = f"models/{model.split('/')[-1]}"

    print(f"\n{'='*60}")
    print(f"=== {model} ===")
    print(f"{'='*60}")

    print(f"\n[1/3] Downloading {model}...")
    !uvx hf download {model} --local-dir={local_dir}

    print(f"\n[2/3] Downloaded files:")
    !du -sh {local_dir}

    print(f"\n[3/3] Uploading to s3://{BUCKET}/{dest_path}...")
    !uvx --from=awscli aws s3 sync {local_dir} s3://{BUCKET}/{dest_path} \
        --region={REGION} \
        --endpoint-url={ENDPOINT_URL}

## Troubleshooting

### Upload failed

If `upload failed` occurs, retry with checksum option:

In [ ]:
# Retry with checksum (for upload failures)
# for model in HF_MODELS:
#     local_dir = f"/content/models/{model}"
#     dest_path = f"models/{model.split('/')[-1]}"
#     !uvx --from=awscli aws s3 sync {local_dir} s3://{BUCKET}/{dest_path} \
#         --region={REGION} \
#         --endpoint-url={ENDPOINT_URL} \
#         --checksum-algorithm=CRC32C

### Sync fails on large volumes

> fatal error: Error during pagination: The same next token was received twice: ...

Use `cp --recursive` instead (no delta transfer):

In [ ]:
# Use cp --recursive if sync fails
# for model in HF_MODELS:
#     local_dir = f"/content/models/{model}"
#     dest_path = f"models/{model.split('/')[-1]}"
#     !uvx --from=awscli aws s3 cp --recursive {local_dir} s3://{BUCKET}/{dest_path} \
#         --region={REGION} \
#         --endpoint-url={ENDPOINT_URL}

In [ ]:
# Verify upload
for model in HF_MODELS:
    dest_path = f"models/{model.split('/')[-1]}"
    print(f"=== {dest_path} ===")
    !uvx --from=awscli aws s3 ls s3://{BUCKET}/{dest_path}/ \
        --region={REGION} \
        --endpoint-url={ENDPOINT_URL}